In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sklearn.metrics

/home/shikha/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
data = pd.read_csv('/content/drive/MyDrive/College/PS - 1/Notebooks/clean_df.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/College/PS - 1/Notebooks/clean_df.csv'

In [ ]:
data.drop(columns=['Unnamed: 0'], inplace = True)

## Model

In [ ]:
model = RandomForestClassifier(n_estimators=10)

In [ ]:
def model_perf(x,y,model):
    cm = metrics.confusion_matrix(y, model.predict(x))
    plt.figure
    sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
    plt.ylabel('Actual label');
    plt.xlabel('Predicted label');
    plt.savefig('covidseverity.png')
    accuracy = model.score(x,y)
    precision,recall,f1,support = sklearn.metrics.precision_recall_fscore_support(y, model.predict(x),average='binary')
    print("Accuracy Score = ",accuracy)
    print("Precision Score = ",precision)    
    print("Recall Score = ",recall)    
    print("F1 Score = ",f1)

## Initial factors

In [ ]:
X = data[['Age Above 60?','Gender','Temp Groups','Respiratory rate(breaths per minute)']]
Y = data['Outcome']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25, random_state=0)

model.fit(Xtrain, ytrain);

In [ ]:
model_perf(x_train, y_train, model)

In [ ]:
model_perf(Xtest, ytest, model)

## Initial Factors + Symptoms and Comorbidities


In [ ]:
X = data[['Age Above 60?','Gender','Temp Groups','Respiratory rate(breaths per minute)','Symptoms Present?','Co-morbidity?','Cough','Breathlessness','Hypertension']]
Y = data['Outcome']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25, random_state=0)

model.fit(Xtrain, ytrain);

In [ ]:
model_perf(Xtrain, ytrain, model)

In [ ]:
model_perf(Xtest, ytest, model)

## Initial Factors + Symptoms and Comorbidities + Blood

In [ ]:
X = data[['Age Above 60?','Gender','Temp Groups','Respiratory rate(breaths per minute)','Symptoms Present?','Co-morbidity?','Cough','Breathlessness','Hypertension','C-REACTIVE PROTEINS','UREA']]
Y = data['Outcome']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25, random_state=0)

model.fit(Xtrain, ytrain);

In [ ]:
model_perf(Xtrain, ytrain, model)

In [ ]:
model_perf(Xtest, ytest, model)

## Future goals - 


1.   Compare accuracies of diff models using boxplots
2.   Make a class for different models

